In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import joblib
import random
import sys

from mip import Model as ModelMip, xsum, minimize, maximize, \
INTEGER, BINARY, CONTINUOUS, CutType, OptimizationStatus

import data_generator
from model import StrongStandardNet, StrongVariationalNet
from train import TrainDecoupled, TrainCombined

from sklearn.preprocessing import StandardScaler

import torch.multiprocessing as mp

In [2]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda') 

In [30]:
method_name = 'bnn'
method_learning = 'combined'
aleat_bool = 1

N_SAMPLES = 16
BATCH_SIZE_LOADER = 64

cpu_count = mp.cpu_count()

seed_number = 0

lr = 0.00001

In [4]:
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

assert (method_name in ['ann','bnn','gp'])

if method_name in ['ann','bnn']:
    assert (method_learning in ['decoupled','combined'])
    assert (aleat_bool in [True, False])
    assert (N_SAMPLES>=1 and N_SAMPLES<9999)
    #assert (M_SAMPLES>=1 and M_SAMPLES<9999)

bnn = False 
if method_name == 'bnn':
    bnn = True   
    K = 1 # Hyperparameter for the training in ELBO loss
    PLV = 1 # Prior in ELBO loss   

model_name = method_name + '_constrained_'
for i in range(2, len(sys.argv)):
    model_name += '_'+sys.argv[i]
model_name += '_'+ str(seed_number)

In [15]:
def gen_intermidiate(num, n_assets, x1, x2, x3):      
        factor = num * 2/(n_assets)
        return x1**factor + x2**factor + x3**factor
    
def gen_data(N, n_assets, nl=1, seed_number=42):
    np.random.seed(seed_number)
    x1 = np.random.normal(1, 1, size = N).clip(0)
    x2 = np.random.normal(1, 1, size = N).clip(0)
    x3 = np.random.normal(1, 1, size = N).clip(0)
    X = np.vstack((x1, x2, x3)).T

    Y = np.zeros((N, n_assets))
    for i in range(1, n_assets + 1):
        interm = gen_intermidiate(i, n_assets, x1, x2, x3)
        Y[:,i-1] = (np.sin(interm) - np.sin(interm).mean()) \
        + np.sin(interm).std()*(-0.3 + np.random.random())
        
        Y[:,i-1] = Y[:,i-1] + nl*Y[:,i-1]*x1*(np.random.beta(2, 5, size = Y[:,0].shape) - 0.2)

    return X, Y

def gen_cond_dist(N, n_assets, n_samples, nl=1, seed_number=420):
    np.random.seed(seed_number)
    Y_dist = np.zeros((n_samples, N, n_assets))
    for i in range(0, n_samples):
        Y_dist[i, :, :] = gen_data(N, n_assets, nl=1, seed_number=np.random.randint(0,999999))[1]
    return Y_dist

In [16]:
class CVaROP():
    def __init__(self, beta, n_assets, min_return, Y_train, Y_dist):
        self.beta = beta
        self.n_assets = n_assets
        self.R = min_return 
        self.uy = Y_train.mean(axis=0)
        
        self.zstar = np.zeros_like(Y_train)
        self.alphastar = np.zeros_like(Y_train[:,0])
        self.cvarstar = np.zeros_like(Y_train[:,0])
        
        for i in range(0, Y_train.shape[0]):
            self.zstar[i,:], _ , self.alphastar[i], self.cvarstar[i] = self.minimize_cvar(Y_dist[:,i,:])
      
    def minimize_cvar(self, y):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0
        z = ([m.add_var(var_type=CONTINUOUS, name=f'z_{i}') for i in range(0, n_assets)])
        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_assets):
            m += z[i] >= 0

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        for i in range(0, n_assets):
            m += xsum(-z[i]*self.uy[i] for i in range(0, n_assets)) <= -self.R

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        zstar = argmins[:n_assets]
        ustar = argmins[n_assets:n_assets+n_samples]
        alphastar = argmins[n_assets+n_samples]    

        return zstar, ustar, alphastar, f_opt
    
    
    def minimize_cvar_given_z(self, y, z):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0

        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        ustar = argmins[:n_samples]
        alphastar = argmins[n_samples]    

        return ustar, alphastar, f_opt
    
    
    def calc_f_per_asset(self, y_pred, y):
        y_pred = self.reshape_outcomes(y_pred) 
        z_star = self.forward(y_pred)
        f_per_item = self.cost_per_item(z_star, y)
        return f_per_item

    def calc_f_per_day(self, y_pred, y):
        f_per_item = self.calc_f_per_asset(y_pred, y)
        f = torch.sum(f_per_item, 1)
        return f

    def cost_fn(self, y_pred, y):
        f = self.calc_f_per_day(y_pred, y)
        f_total = torch.mean(f)
        return f_total
        
    def end_loss(self, y_pred, y):
        y_pred = y_pred.unsqueeze(0)
        f_total = self.cost_fn(y_pred, y)
        return f_total
    
    def end_loss_dist(self, y_pred, y):
        f_total = self.cost_fn(y_pred, y)
        return f_total

In [45]:
N_train = 1000
N_val = 1000
N_test = 3000

n_assets = 20

n_samples_orig = 200

X, Y_original = gen_data(N_train, n_assets, seed_number)
X_val, Y_val_original = gen_data(N_val, n_assets, seed_number + 80)
X_test, Y_test_original = gen_data(N_test, n_assets, seed_number + 160)

Y_dist = gen_cond_dist(N_train, n_assets, n_samples_orig, nl=1, seed_number=seed_number)
Y_val_dist = gen_cond_dist(N_val, n_assets, n_samples_orig, nl=1, seed_number=seed_number + 100)
Y_test_dist = gen_cond_dist(N_test, n_assets, n_samples_orig, nl=1, seed_number=seed_number + 200)

In [46]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_).to(dev)
tstd = torch.tensor(scaler.scale_).to(dev)
joblib.dump(scaler, 'scaler_cvar.gz')

# Function to denormalize the data
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)
data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=True, num_workers=cpu_count)
Y_dist = torch.tensor(Y_dist, dtype=torch.float32)

Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)
data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)

X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test_original = torch.tensor(
    Y_test_original, dtype=torch.float32)
data_test = data_generator.ArtificialDataset(
    X_test, Y_test_original)
test_loader = torch.utils.data.DataLoader(
data_test, batch_size=16,
shuffle=False, num_workers=cpu_count)

In [47]:
n_samples = Y.shape[0]
n_assets = Y.shape[1]
beta = 0.95
min_return = 10

In [51]:
op_solver_dist = CVaROP(beta, n_assets, min_return, Y_original, Y_dist.detach().numpy())

In [52]:
if method_name == 'bnn':
    h = StrongVariationalNet(
    n_samples=N_SAMPLES,
    input_size=X.shape[1], 
    output_size=Y.shape[1], 
    plv=PLV, 
    dev=dev).to(dev)

#ANN Baseline model
elif method_name == 'ann':
    h = StrongStandardNet(X.shape[1], Y.shape[1]).to(dev)
    K = 0 # There is no K in ANN

In [53]:
opt_h = torch.optim.Adam(h.parameters(), lr=lr)
mse_loss = nn.MSELoss(reduction='none')

In [55]:
# Decoupled learning approach
if method_learning == 'decoupled':
    train_NN = TrainDecoupled(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    loss_data=mse_loss,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    validation_loader=validation_loader,
                    dev=dev
                )

# Combined learning approach (end-to-end loss)
elif method_learning == 'combined':
    train_NN = TrainCombined(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    scaler=scaler,
                    validation_loader=validation_loader,
                    OP=op_solver_dist,
                    dev=dev
                )

AttributeError: 'CVaROP' object has no attribute 'end_loss'

In [78]:
Y_pred = np.random.random((50, N_train, n_assets)) - 5

In [79]:
zstar_pred = np.zeros_like(Y_pred[0])

for i in range(0, Y_pred.shape[1]):
    zstar_pred[i,:] , _, _, _ = op_solver.minimize_cvar(Y_pred[:,i,:])

In [80]:
alphastar_pred = np.zeros_like(Y_pred[0,:,0])
for i in range(0, Y_pred.shape[1]):
    _, alphastar_pred[i] , _ = op_solver.minimize_cvar_given_z(Y_dist[:,i,:], zstar_pred[i,:])

In [81]:
cvar_pred = alphastar_pred + (1/(1-beta))*(np.maximum(-((Y_dist*zstar_pred).sum(2)) - alphastar_pred, 0)).mean(0)

In [82]:
f_pred_dataset = cvar_pred.mean()

In [83]:
fair_regret = f_pred_dataset - op_solver.cvarstar.mean()

In [84]:
fair_regret

4.802389564712255

In [15]:
op_solver.alphastar + (1/(1-beta))*(np.maximum(-((Y_dist*op_solver.zstar).sum(2)) - op_solver.alphastar, 0)).mean(0)

array([14.53573779, 15.40195475, 16.40051164, 15.38380813, 14.62208475,
       17.06472651, 16.73093918, 15.48154203, 15.9300598 , 16.2041831 ,
       15.73938929, 16.89708193, 15.03783049, 15.81537838, 16.31737609,
       15.29510803, 16.09471929, 14.65658093, 15.2551622 , 15.96507641,
       17.64164752, 17.32653127, 14.25618879, 13.61590662, 16.92963039,
       15.39177444, 15.94804965, 15.49853349, 17.32618356, 15.23503582,
       15.87627014, 15.36836605, 16.97589351, 14.78739717, 16.26122729,
       18.07661054, 15.84807938, 14.6334357 , 16.64567533, 15.43718012,
       15.94837282, 14.39151073, 16.18164058, 14.63956316, 16.72973545,
       17.41537503, 15.88181541, 16.850747  , 15.30816212, 13.10728099,
       13.95678749, 15.29048891, 15.62895434, 16.26142701, 15.8916077 ,
       15.7978594 , 16.34415953, 15.10308489, 15.14827456, 15.24617177,
       16.37071953, 15.95829432, 14.10051356, 14.39860641, 14.34294722,
       15.74917112, 14.66477814, 15.33982245, 16.17635938, 14.59

In [ ]:
def compute_cvar(alpha, z, y):
    

In [154]:
1/(Y_dist.shape[0]*(1-beta)) * Y_dist

(1000, 1000, 20)

In [ ]:
Y_dist

In [159]:
Y_dist*zstar

array([[[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00, -8.01762101e-02,  0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00,  7.26688064e-01],
        [-2.77114335e-01, -3.74680090e-01, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -3.38799630e+00],
        ...,
        [-1.75644780e-02, -2.58917253e+00, -0.00000000e+00, ...,
          5.05400752e+00, -3.07404112e-01, -3.07341864e+00],
        [-5.71981859e-01, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00,  1.60096702e-01,  0.00000000e+00],
        [-3.02333994e-02, -0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  2.50841331e+00,  7.44609135e-02]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.62525526e-01,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e

In [155]:
Y_dist.shape[0]

1000

In [151]:
Y_pred.shape

(50, 1000, 20)

In [137]:
Y_pred

array([[[-0.17638682, -0.09089706, -0.3830332 , ...,  0.06720436,
          0.42834576,  0.26608252],
        [ 0.32939309,  0.55933907,  0.0138017 , ..., -0.38775832,
          0.38721256,  0.48453956],
        [-0.08600242, -0.06805484,  0.51201416, ..., -0.30176222,
         -0.30011163,  0.48041784],
        ...,
        [ 0.5547552 ,  0.53806358,  0.41646602, ..., -0.1646881 ,
         -0.38714016,  0.0585669 ],
        [-0.259434  , -0.03745498, -0.23963089, ...,  0.41290765,
          0.06538298, -0.25733343],
        [ 0.1752564 ,  0.55979906, -0.1785693 , ...,  0.22204114,
          0.43828789, -0.28569533]],

       [[ 0.25746872, -0.39684173, -0.3284593 , ...,  0.3906845 ,
         -0.39183591,  0.16925576],
        [-0.01981441, -0.05351203,  0.32120704, ...,  0.0888863 ,
          0.13828446,  0.25011441],
        [-0.23420105, -0.14614686,  0.52928029, ...,  0.4170639 ,
          0.55037046,  0.51355313],
        ...,
        [ 0.54040521,  0.23890133, -0.31902953, ..., -

In [8]:
Y_train[:50].mean(0)

array([ 0.01103508,  0.18582536,  0.02757531,  0.176867  ,  0.28287247,
        0.24011691,  0.10373269,  0.22923871,  0.30873738, -0.07671305,
        0.39660958, -0.21470825,  0.23993704, -0.19205835,  0.06976262,
       -0.07803554,  0.40508948, -0.03724071,  0.02091073,  0.40063581])

In [9]:
Y_train[:50].std(0)

array([0.41032437, 0.46137744, 0.52203211, 0.58222118, 0.63199747,
       0.66359498, 0.67581864, 0.67831638, 0.68665954, 0.70163281,
       0.70021433, 0.67717812, 0.68244412, 0.71339072, 0.69029142,
       0.6841247 , 0.67276091, 0.62925573, 0.6156747 , 0.7166073 ])

In [10]:
op_solver.minimize_cvar(Y_train[:50])

([0.0,
  0.0,
  0.0,
  0.0,
  10.93680879473024,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.475523075674423,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  4.652755334066428,
  0.0,
  0.0,
  3.8449485359989164],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.27095111058335786,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 3.958868157503432,
 4.067248601736775)

In [34]:
y = Y_train

In [133]:
y.mean(axis=0)

array([ 0.09154903,  0.12018547, -0.00612775,  0.08615439,  0.07785487])

In [125]:
y.std(axis=0)

array([0.28204021, 0.29966942, 0.2348671 , 0.25421325, 0.31059858])

In [126]:
m.status.name

'OPTIMAL'

In [127]:
alpha_opt = -1

In [128]:
(1/n_samples)*(1/(1-beta))*np.sum(np.clip(((-y*np.array(argmins[:5])).sum(axis=1) - alpha_opt), 0, None))

71.47976078082966

In [129]:
(y.mean(axis=0)*np.array(argmins[:5])).sum()

9.999999999999998

In [134]:
argmins

[53.23569452514242,
 18.33912369004899,
 0.0,
 30.08292387425699,
 4.372902220899407,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.4596077782392385,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 17.568190336620138]

In [135]:
f_opt

18.152033447915834

In [3]:
(1/(1 - B))*(1/n)

1.9999999999999982

In [ ]:


class LPKnapsackProblem():
    def __init__(self, Y, probs_mean):
        self.Y = Y
        self.probs_mean = probs_mean
        self.n_items = self.Y.shape[1]
        self.weights, self.capacity = self.define_knapsack()
        
    def define_knapsack(self):
        weights = self.Y.mean(axis=0)*self.probs_mean + 80*np.random.random(size=self.n_items)
        capacity = np.random.uniform(0.4, 0.6) * sum(weights)
        return weights, int(capacity)
    
    def solve_knapsack_mip(self, values, activations):
        m = ModelMip("knapsack")
        m.verbose = 0
        z = ([m.add_var(var_type=CONTINUOUS) for i in range(0, self.n_items)])
        m.objective = minimize(
            -xsum(values[i] * z[i] for i in range(0, self.n_items)))
        
        for i in range(0, self.n_items):
            m += z[i] <= 1
            m += -z[i] <= 0
            
        m += xsum(
            self.weights[i] * z[i] for i in range(0, self.n_items)) <= self.capacity
        m += xsum(z[i] for i in range(0, self.n_items) if activations[i]==0) <= 0
        
        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
        return argmins, f_opt